In [1]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score,classification_report
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas

In [2]:
Xtr = np.load("train_features.npy")
ytr = np.load("train_labels.npy")
num_classes = np.max(ytr) + 1

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size=num_classes, dropout_prob=0.5):
        super(MLP, self).__init__()
        
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.out = nn.Linear(64, output_size)
        
        self.dropout = nn.Dropout(p=dropout_prob)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        
        x = self.out(x)
        return x

In [ ]:
class MLP_Simple(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=128, dropout_prob=0.1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.out(x)
        return x


In [3]:
# Example: medium model
class MLP_Medium(nn.Module):
    def __init__(self, input_size, output_size, hidden1=512, hidden2=256, dropout_prob=0.2):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.dropout = nn.Dropout(p=dropout_prob)
        self.out = nn.Linear(hidden2, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.out(x)
        return x


In [4]:
num_epochs=100
lr=0.001
dropout=0.2
batch_size=64

In [5]:
Xtr = torch.tensor(Xtr, dtype=torch.float32)
ytr = torch.tensor(ytr, dtype=torch.long)
#Xte = torch.tensor(Xte, dtype=torch.float32)
#yte = torch.tensor(yte, dtype=torch.float32)

# Wrap Xtr and ytr into a dataset
train_dataset = TensorDataset(Xtr, ytr)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP(input_size=Xtr.shape[1], dropout_prob=dropout).to(device)
criterion = nn.CrossEntropyLoss()  # for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP_Simple(input_size=Xtr.shape[1], output_size=num_classes, hidden_size=128, dropout_prob=0.1).to(device)
criterion = nn.CrossEntropyLoss()  # for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=lr)

In [6]:
# Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP_Medium(input_size=Xtr.shape[1], output_size=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=lr)

In [7]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for batch_x, batch_y in train_dataloader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        logits = model(batch_x)
        loss = criterion(logits, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/100], Loss: 3.1461
Epoch [2/100], Loss: 1.9866
Epoch [3/100], Loss: 1.9669
Epoch [4/100], Loss: 1.9604
Epoch [5/100], Loss: 1.9480
Epoch [6/100], Loss: 1.9383
Epoch [7/100], Loss: 1.9431
Epoch [8/100], Loss: 1.9315
Epoch [9/100], Loss: 1.9384
Epoch [10/100], Loss: 1.9389
Epoch [11/100], Loss: 1.9365
Epoch [12/100], Loss: 1.9219
Epoch [13/100], Loss: 1.9295
Epoch [14/100], Loss: 1.9251
Epoch [15/100], Loss: 1.9268
Epoch [16/100], Loss: 1.9356
Epoch [17/100], Loss: 1.9313
Epoch [18/100], Loss: 1.9297
Epoch [19/100], Loss: 1.9469
Epoch [20/100], Loss: 1.9450
Epoch [21/100], Loss: 1.9103
Epoch [22/100], Loss: 1.9327
Epoch [23/100], Loss: 1.9382
Epoch [24/100], Loss: 1.9386
Epoch [25/100], Loss: 1.9268
Epoch [26/100], Loss: 1.9301
Epoch [27/100], Loss: 1.9401
Epoch [28/100], Loss: 1.9262
Epoch [29/100], Loss: 2.1464
Epoch [30/100], Loss: 2.1300
Epoch [31/100], Loss: 2.1310
Epoch [32/100], Loss: 2.1303
Epoch [33/100], Loss: 2.1314
Epoch [34/100], Loss: 2.1298


KeyboardInterrupt: 

In [ ]:
Xtest_studio = np.load("test_studio_features.npy")
ytest_studio = np.load("test_studio_labels.npy")

Xtest_studio = torch.tensor(Xtest_studio, dtype=torch.float32)
ytest_studio = torch.tensor(ytest_studio , dtype=torch.long)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Xtest_studio = Xtest_studio.to(device)
ytest_studio = ytest_studio.to(device)

model = model.to(device)
model.eval()  # set model to evaluation mode


with torch.no_grad():  # disable gradient computation
    logits = model(Xtest_studio)                 # forward pass
    preds = torch.argmax(logits, dim=1)          # get class indices


from sklearn.metrics import accuracy_score, classification_report

y_true = ytest_studio.cpu().numpy()  # true labels
y_pred = preds.cpu().numpy()         # predicted labels

# Overall accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Full classification report (precision, recall, F1-score per class)
report = classification_report(y_true, y_pred)
print(report)






In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Xtr = Xtr.to(device)
ytr = ytr.to(device)

model = model.to(device)
model.eval()  # set model to evaluation mode


with torch.no_grad():  # disable gradient computation
    logits = model(Xtr)                 # forward pass
    preds = torch.argmax(logits, dim=1)          # get class indices


from sklearn.metrics import accuracy_score, classification_report

y_true = ytr.cpu().numpy()  # true labels
y_pred = preds.cpu().numpy()         # predicted labels

# Overall accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Full classification report (precision, recall, F1-score per class)
report = classification_report(y_true, y_pred)
print(report)

In [ ]:
y_pred=model(Xtest_studio)
print(f'ACC:{accuracy_score(ytest_studio.detach().numpy(),y_pred.detach().numpy()>0.5)}') #classification
#print(f'MSE:{mean_squared_error(yte.detach().numpy(),y_pred.detach().numpy())}') #regression